In [1]:
# activate the project environment
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."));

  Activating project at `~/projects/julia-projects/dbscan`


In [6]:
using NearestNeighbors
using dbscan
using StaticArrays
using Dates

In [3]:
# points = [SVector{3}(rand(3)) for i in 1:1_000_000]
# points = map(i -> rand(3), 1:1_000_000)
# points = [rand(3) for _ in 1:1_000_000]
points = rand(SVector{3, Float64}, 1_000_000)

1000000-element Vector{SVector{3, Float64}}:
 [0.8978414181071017, 0.2334835651968209, 0.4961077000730659]
 [0.9375717161530257, 0.338701768569888, 0.939861704758527]
 [0.2895604154793344, 0.03735108468063664, 0.20991596477214935]
 [0.10238764592592708, 0.5021790120897107, 0.18646212611173807]
 [0.30338850582375576, 0.26839223385844246, 0.5238345503654466]
 [0.3285232961036183, 0.04390640024549253, 0.9951814106206902]
 [0.7043175833567811, 0.11663314841775019, 0.6575408397566688]
 [0.8289846940061493, 0.673626445765438, 0.48517362809856324]
 [0.8905295840839083, 0.618145811081623, 0.3729209899360707]
 [0.9952859570454994, 0.6077366083104163, 0.7463926728682917]
 ⋮
 [0.07990364414343132, 0.9193200370273642, 0.3735910972114035]
 [0.990085338800328, 0.4174843568704997, 0.13646045007686713]
 [0.8240903123647852, 0.6169725156427633, 0.8682500649736019]
 [0.3280941402206391, 0.021324179943215116, 0.5171733437979027]
 [0.8310022384123119, 0.8039810270305947, 0.15781465188165855]
 [0.776919879

In [4]:
dbscan.DBSCAN(points, 0.01, 3)

1000000-element Vector{Int64}:
 814699
 707049
 475408
 889542
 961030
 882536
 988987
 849405
 824324
 962854
      ⋮
 999992
 999993
 999994
      0
 999996
 999997
 999998
 999999
      0

In [10]:
function run_tests(N)
    times = zeros(N)
    for i in 1:N
        points = rand(SVector{3, Float64}, 1_000_000)
        t0 = now()
        labels = dbscan.DBSCAN(points, 0.01, 3)
        tf = now()
        times[i] = (tf - t0).value / 1000
    end
    return times
end

run_tests (generic function with 1 method)

In [12]:
times = run_tests(10)

10-element Vector{Float64}:
 1.254
 1.286
 1.264
 1.208
 1.184
 1.238
 1.169
 1.14
 1.174
 1.291

In [ ]:
# smaller dataset for plotting